In [ ]:
!nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

Mon Sep 19 00:42:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# CUDA version:
!nvcc -V
# Check GCC version:
!gcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

cu_version='cu111'
torch_version='torch1.10'

!pip install mmcv-full==1.6.0 -f https://download.openmmlab.com/mmcv/dist/{cu_version}/{torch_version}/index.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 1.4 MB/s eta 0:15:09tcmalloc: large alloc 1147494400 bytes == 0x39534000 @  0x7fd166c3a615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |███████████████▉                | 1055.7 MB 1.5 MB/s eta 0:11:53tcmalloc: large alloc 1434370048 bytes == 0x7db8a000 @  0x7fd166c3a615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |█████

In [ ]:
%cd /content
!rm -rf mmsegmentation
!git clone https://github.com/ykawa2/mmsegmentation.git
%cd mmsegmentation
!pip install -e . 

!python mmseg/utils/collect_env.py

/content
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 7677, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7677 (delta 0), reused 4 (delta 0), pack-reused 7671
Receiving objects: 100% (7677/7677), 13.94 MiB | 3.55 MiB/s, done.
Resolving deltas: 100% (5671/5671), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
     |████████████████████████████████| 578 kB 3.4 MB/s 
  Running setup.py develop for mmsegmentation
sys.platform: linux
Python: 3.7.14 (default, Sep  8 2022, 00:06:44) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.10.0+cu111
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - In

In [ ]:
# Check Pytorch installation
import torch, torchvision
print('torch version:', torch.__version__, 'cuda_availabe:', torch.cuda.is_available())
print('torchvision version:', torchvision.__version__) 

import mmseg
print('mmseg version:', mmseg.__version__) 

import mmcv
print('mmcv version:', mmcv.__version__)

torch version: 1.10.0+cu111 cuda_availabe: True
torchvision version: 0.11.0+cu111
mmseg version: 0.27.0
mmcv version: 1.6.0


## Dataset

In [ ]:
%cd /content
!tar -xvf /content/drive/MyDrive/kaggle/multi_class_dataset/hubmap_2000x2000_aug_v5.tar > /dev/null
!mv /content/hubmap_2000x2000 /content/hubmap_multi_2000x2000

/content


# Merge of mask patch

In [ ]:
!cp -r /content/drive/MyDrive/kaggle/multi_class_dataset/converted_mask_patch_v3 /content

In [ ]:
# converted_mask_patchにあるpngファイル名と同一名のpngファイルとマスクのマージを行う
import glob
import os
from PIL import Image
import numpy as np

target_index=[203,676,3409,4802,5287,8222,8402,9231,12174,12471,14183,15732,16659,
            20247,24194,28318,28622,29180,32412,29307, 8388,17143, 19569, 5317]
replace_index=[]
#replace_index=[29307, 8388]
#no_target=[17143, 19569, 5317]

patch_dir='/content/converted_mask_patch_v3'
mask_dir='/content/hubmap_multi_2000x2000/lung_refined_masks'


files=glob.glob(patch_dir + '/*.png')
basename_without_ext=[os.path.basename(f).split('.')[0] for f in files ]

cnt=0
for idx in basename_without_ext:
    if int(idx) in target_index:
        patch_path=os.path.join(patch_dir, idx + '.png')
        mask_path=os.path.join(mask_dir, idx + '.png')

        mask=Image.open(mask_path)
        assert mask.mode=='P'
        palette=mask.getpalette()
        mask=np.array(mask)

        patch=Image.open(patch_path)
        assert patch.mode=='P'
        patch=np.array(patch)

        assert mask.shape==patch.shape
        
        mask[patch>0]=0
        mask+=patch

        mask=Image.fromarray(mask)
        mask.putpalette(palette)
        mask.save(mask_path)

    elif int(idx) in replace_index:
        patch_path=os.path.join(patch_dir, idx + '.png')
        mask_path=os.path.join(mask_dir, idx + '.png')

        patch=Image.open(patch_path)
        assert patch.mode=='P'
        
        patch.save(mask_path)
    
    else:
        continue

    cnt+=1
    print(mask_path)

print(cnt)


/content/hubmap_multi_2000x2000/lung_refined_masks/29180.png
/content/hubmap_multi_2000x2000/lung_refined_masks/8388.png
/content/hubmap_multi_2000x2000/lung_refined_masks/5287.png
/content/hubmap_multi_2000x2000/lung_refined_masks/676.png
/content/hubmap_multi_2000x2000/lung_refined_masks/17143.png
/content/hubmap_multi_2000x2000/lung_refined_masks/28622.png
/content/hubmap_multi_2000x2000/lung_refined_masks/12174.png
/content/hubmap_multi_2000x2000/lung_refined_masks/15732.png
/content/hubmap_multi_2000x2000/lung_refined_masks/5317.png
/content/hubmap_multi_2000x2000/lung_refined_masks/32412.png
/content/hubmap_multi_2000x2000/lung_refined_masks/3409.png
/content/hubmap_multi_2000x2000/lung_refined_masks/24194.png
/content/hubmap_multi_2000x2000/lung_refined_masks/28318.png
/content/hubmap_multi_2000x2000/lung_refined_masks/29307.png
/content/hubmap_multi_2000x2000/lung_refined_masks/12471.png
/content/hubmap_multi_2000x2000/lung_refined_masks/19569.png
/content/hubmap_multi_2000x200

# 2classデータセットの生成

In [ ]:
!cp -r /content/hubmap_multi_2000x2000/lung_refined_masks /content/hubmap_multi_2000x2000/lung_refined_masks_2class

In [ ]:
import glob
from PIL import Image
import numpy as np

from mmseg.mylib.seg_utils import convert_rgb_to_voc_palette

files=glob.glob('/content/hubmap_multi_2000x2000/lung_refined_masks_2class/*.png')

for idx, f in enumerate(files):
    pil_mask=Image.open(f)
    assert pil_mask.mode=='P'

    palette=pil_mask.getpalette()

    mask=np.asarray(pil_mask)
    mask=np.where(mask>=1, 1, 0)
    mask=mask.astype(np.uint8)

    new_mask=Image.fromarray(mask)
    new_mask.putpalette(palette)
    assert new_mask.mode=='P', new_mask.mode
    unique=list(np.unique(np.asarray(new_mask)))
    assert unique==[0,1] or unique==[0], unique
    new_mask.save(f)

    print(idx, f)

0 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/30424.png
1 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/30500.png
2 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/13396.png
3 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/2344.png
4 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/19084.png
5 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/19360.png
6 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/6722.png
7 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/21321.png
8 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/27232.png
9 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/2874.png
10 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/9407.png
11 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/10392.png
12 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/27781.png
13 /content/hubmap_multi_2000x2000/lung_refined_masks_2class/4062.png
14 /content/hubmap_mu

# external spleen dataset

In [ ]:
%cd /content
!tar -xvf /content/drive/MyDrive/kaggle/multi_class_dataset/external_spleen_v2.tar

/content
external_spleen_v2/
external_spleen_v2/image/
external_spleen_v2/image/stain4_24.png
external_spleen_v2/image/stain3_4.png
external_spleen_v2/image/stain4_17.png
external_spleen_v2/image/25.png
external_spleen_v2/image/stain4_14.png
external_spleen_v2/image/stain3_0.png
external_spleen_v2/image/stain3_22.png
external_spleen_v2/image/stain2_18.png
external_spleen_v2/image/stain1_15.png
external_spleen_v2/image/stain2_15.png
external_spleen_v2/image/stain1_1.png
external_spleen_v2/image/stain1_0.png
external_spleen_v2/image/stain3_31.png
external_spleen_v2/image/stain2_24.png
external_spleen_v2/image/stain3_29.png
external_spleen_v2/image/stain3_12.png
external_spleen_v2/image/stain1_18.png
external_spleen_v2/image/stain2_37.png
external_spleen_v2/image/stain2_2.png
external_spleen_v2/image/stain4_12.png
external_spleen_v2/image/stain3_7.png
external_spleen_v2/image/6.png
external_spleen_v2/image/stain2_20.png
external_spleen_v2/image/stain4_28.png
external_spleen_v2/image/33.pn

## external lung dataset

In [ ]:
%cd /content
!tar -xvf /content/drive/MyDrive/kaggle/multi_class_dataset/external_lung_v1.tar

/content
external_lung_v1/
external_lung_v1/image/
external_lung_v1/image/lung_39.png
external_lung_v1/image/lung_77.png
external_lung_v1/image/lung_49.png
external_lung_v1/image/lung_81.png
external_lung_v1/image/lung_72.png
external_lung_v1/image/lung_73.png
external_lung_v1/image/lung_66.png
external_lung_v1/image/lung_32.png
external_lung_v1/image/lung_22.png
external_lung_v1/image/lung_46.png
external_lung_v1/image/lung_63.png
external_lung_v1/image/lung_60.png
external_lung_v1/image/lung_70.png
external_lung_v1/image/lung_0.png
external_lung_v1/image/lung_51.png
external_lung_v1/image/lung_59.png
external_lung_v1/image/lung_45.png
external_lung_v1/image/lung_36.png
external_lung_v1/image/lung_61.png
external_lung_v1/image/lung_31.png
external_lung_v1/image/lung_14.png
external_lung_v1/image/lung_37.png
external_lung_v1/image/lung_30.png
external_lung_v1/image/lung_20.png
external_lung_v1/image/lung_25.png
external_lung_v1/image/lung_56.png
external_lung_v1/image/lung_52.png
exter

In [ ]:
!pip install -q wandb

import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 12.0 MB/s 
     |████████████████████████████████| 158 kB 48.8 MB/s 
     |████████████████████████████████| 181 kB 40.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 157 kB 49.2 MB/s 
     |████████████████████████████████| 157 kB 51.5 MB/s 
     |████████████████████████████████| 157 kB 50.4 MB/s 
     |████████████████████████████████| 157 kB 48.4 MB/s 
     |████████████████████████████████| 157 kB 55.4 MB/s 
     |████████████████████████████████| 157 kB 51.3 MB/s 
     |████████████████████████████████| 157 kB 52.4 MB/s 
     |████████████████████████████████| 156 kB 50.6 MB/s 


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## mmsegmentation mDice settings

In [ ]:
%%bash
cat << EOF > /content/mmsegmentation/mmseg/core/evaluation/metrics.py
# Copyright (c) OpenMMLab. All rights reserved.
from collections import OrderedDict

import mmcv
import numpy as np
import torch


def f_score(precision, recall, beta=1):
    """calculate the f-score value.

    Args:
        precision (float | torch.Tensor): The precision value.
        recall (float | torch.Tensor): The recall value.
        beta (int): Determines the weight of recall in the combined score.
            Default: False.

    Returns:
        [torch.tensor]: The f-score value.
    """
    score = (1 + beta**2) * (precision * recall) / (
        (beta**2 * precision) + recall)
    return score


def intersect_and_union(pred_label,
                        label,
                        num_classes,
                        ignore_index,
                        label_map=dict(),
                        reduce_zero_label=False):
    """Calculate intersection and Union.

    Args:
        pred_label (ndarray | str): Prediction segmentation map
            or predict result filename.
        label (ndarray | str): Ground truth segmentation map
            or label filename.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        label_map (dict): Mapping old labels to new labels. The parameter will
            work only when label is str. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. The parameter will
            work only when label is str. Default: False.

     Returns:
         torch.Tensor: The intersection of prediction and ground truth
            histogram on all classes.
         torch.Tensor: The union of prediction and ground truth histogram on
            all classes.
         torch.Tensor: The prediction histogram on all classes.
         torch.Tensor: The ground truth histogram on all classes.
    """

    if isinstance(pred_label, str):
        pred_label = torch.from_numpy(np.load(pred_label))
    else:
        pred_label = torch.from_numpy((pred_label))

    if isinstance(label, str):
        label = torch.from_numpy(
            mmcv.imread(label, flag='unchanged', backend='pillow'))
    else:
        label = torch.from_numpy(label)

    if label_map is not None:
        for old_id, new_id in label_map.items():
            label[label == old_id] = new_id
    if reduce_zero_label:
        label[label == 0] = 255
        label = label - 1
        label[label == 254] = 255

    mask = (label != ignore_index)
    pred_label = pred_label[mask]
    label = label[mask]

    intersect = pred_label[pred_label == label]
    area_intersect = torch.histc(
        intersect.float(), bins=(num_classes), min=0, max=num_classes - 1)
    area_pred_label = torch.histc(
        pred_label.float(), bins=(num_classes), min=0, max=num_classes - 1)
    area_label = torch.histc(
        label.float(), bins=(num_classes), min=0, max=num_classes - 1)
    area_union = area_pred_label + area_label - area_intersect
    return area_intersect, area_union, area_pred_label, area_label


def total_intersect_and_union(results,
                              gt_seg_maps,
                              num_classes,
                              ignore_index,
                              label_map=dict(),
                              reduce_zero_label=False,
                              ):
    """Calculate Total Intersection and Union.

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str] | Iterables): list of ground
            truth segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.

     Returns:
         ndarray: The intersection of prediction and ground truth histogram
             on all classes.
         ndarray: The union of prediction and ground truth histogram on all
             classes.
         ndarray: The prediction histogram on all classes.
         ndarray: The ground truth histogram on all classes.
    """

    print('excuting <total_intersect_and_union>')
    total_area_intersect = torch.zeros((num_classes, ), dtype=torch.float64)
    total_area_union = torch.zeros((num_classes, ), dtype=torch.float64)
    total_area_pred_label = torch.zeros((num_classes, ), dtype=torch.float64)
    total_area_label = torch.zeros((num_classes, ), dtype=torch.float64)
    per_img_mdice = torch.zeros((num_classes, ), dtype=torch.float64)

    cnt = 0
    for result, gt_seg_map in zip(results, gt_seg_maps):
        area_intersect, area_union, area_pred_label, area_label = \
            intersect_and_union(
                result, gt_seg_map, num_classes, ignore_index,
                label_map, reduce_zero_label)
        total_area_intersect += area_intersect
        total_area_union += area_union
        total_area_pred_label += area_pred_label
        total_area_label += area_label

        dice_coefficient = 2 * area_intersect / (area_pred_label + area_label)
        per_img_mdice += dice_coefficient
        cnt += 1

    per_img_mdice /= cnt
    return total_area_intersect, total_area_union, total_area_pred_label, \
        total_area_label, per_img_mdice


def mean_iou(results,
             gt_seg_maps,
             num_classes,
             ignore_index,
             nan_to_num=None,
             label_map=dict(),
             reduce_zero_label=False):
    """Calculate Mean Intersection and Union (mIoU)

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str]): list of ground truth
            segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.

     Returns:
        dict[str, float | ndarray]:
            <aAcc> float: Overall accuracy on all images.
            <Acc> ndarray: Per category accuracy, shape (num_classes, ).
            <IoU> ndarray: Per category IoU, shape (num_classes, ).
    """
    iou_result = eval_metrics(
        results=results,
        gt_seg_maps=gt_seg_maps,
        num_classes=num_classes,
        ignore_index=ignore_index,
        metrics=['mIoU'],
        nan_to_num=nan_to_num,
        label_map=label_map,
        reduce_zero_label=reduce_zero_label)
    return iou_result


def mean_dice(results,
              gt_seg_maps,
              num_classes,
              ignore_index,
              nan_to_num=None,
              label_map=dict(),
              reduce_zero_label=False):
    """Calculate Mean Dice (mDice)

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str]): list of ground truth
            segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.

     Returns:
        dict[str, float | ndarray]: Default metrics.
            <aAcc> float: Overall accuracy on all images.
            <Acc> ndarray: Per category accuracy, shape (num_classes, ).
            <Dice> ndarray: Per category dice, shape (num_classes, ).
    """

    dice_result = eval_metrics(
        results=results,
        gt_seg_maps=gt_seg_maps,
        num_classes=num_classes,
        ignore_index=ignore_index,
        metrics=['mDice'],
        nan_to_num=nan_to_num,
        label_map=label_map,
        reduce_zero_label=reduce_zero_label)
    return dice_result


def mean_fscore(results,
                gt_seg_maps,
                num_classes,
                ignore_index,
                nan_to_num=None,
                label_map=dict(),
                reduce_zero_label=False,
                beta=1):
    """Calculate Mean F-Score (mFscore)

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str]): list of ground truth
            segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.
        beta (int): Determines the weight of recall in the combined score.
            Default: False.


     Returns:
        dict[str, float | ndarray]: Default metrics.
            <aAcc> float: Overall accuracy on all images.
            <Fscore> ndarray: Per category recall, shape (num_classes, ).
            <Precision> ndarray: Per category precision, shape (num_classes, ).
            <Recall> ndarray: Per category f-score, shape (num_classes, ).
    """
    fscore_result = eval_metrics(
        results=results,
        gt_seg_maps=gt_seg_maps,
        num_classes=num_classes,
        ignore_index=ignore_index,
        metrics=['mFscore'],
        nan_to_num=nan_to_num,
        label_map=label_map,
        reduce_zero_label=reduce_zero_label,
        beta=beta)
    return fscore_result


def eval_metrics(results,
                 gt_seg_maps,
                 num_classes,
                 ignore_index,
                 metrics=['mIoU'],
                 nan_to_num=None,
                 label_map=dict(),
                 reduce_zero_label=False,
                 beta=1):
    """Calculate evaluation metrics
    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str] | Iterables): list of ground
            truth segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        metrics (list[str] | str): Metrics to be evaluated, 'mIoU' and 'mDice'.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.
     Returns:
        float: Overall accuracy on all images.
        ndarray: Per category accuracy, shape (num_classes, ).
        ndarray: Per category evaluation metrics, shape (num_classes, ).
    """

    total_area_intersect, total_area_union, total_area_pred_label, \
        total_area_label, per_img_mdice = total_intersect_and_union(
            results, gt_seg_maps, num_classes, ignore_index, label_map,
            reduce_zero_label)
    ret_metrics = total_area_to_metrics(total_area_intersect, total_area_union,
                                        total_area_pred_label,
                                        total_area_label,
                                        per_img_mdice,
                                        metrics, nan_to_num,
                                        beta)

    return ret_metrics


def pre_eval_to_metrics(pre_eval_results,
                        metrics=['mIoU'],
                        nan_to_num=None,
                        beta=1):
    """Convert pre-eval results to metrics.

    Args:
        pre_eval_results (list[tuple[torch.Tensor]]): per image eval results
            for computing evaluation metric
        metrics (list[str] | str): Metrics to be evaluated, 'mIoU' and 'mDice'.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
     Returns:
        float: Overall accuracy on all images.
        ndarray: Per category accuracy, shape (num_classes, ).
        ndarray: Per category evaluation metrics, shape (num_classes, ).
    """

    # convert list of tuples to tuple of lists, e.g.
    # [(A_1, B_1, C_1, D_1), ...,  (A_n, B_n, C_n, D_n)] to
    # ([A_1, ..., A_n], ..., [D_1, ..., D_n])
    print('\nexcuting <pre_eval_to_metrics>')
    pre_eval_results = tuple(zip(*pre_eval_results))
    assert len(pre_eval_results) == 4

    total_area_intersect = sum(pre_eval_results[0])
    total_area_union = sum(pre_eval_results[1])
    total_area_pred_label = sum(pre_eval_results[2])
    total_area_label = sum(pre_eval_results[3])
    per_img_mdice = None  # sum(pre_eval_results[4])

    ret_metrics = total_area_to_metrics(total_area_intersect, total_area_union,
                                        total_area_pred_label,
                                        total_area_label,
                                        per_img_mdice,
                                        metrics, nan_to_num,
                                        beta)

    return ret_metrics


def total_area_to_metrics(total_area_intersect,
                          total_area_union,
                          total_area_pred_label,
                          total_area_label,
                          per_img_mdice,
                          metrics=['mIoU'],
                          nan_to_num=None,
                          beta=1):
    """Calculate evaluation metrics
    Args:
        total_area_intersect (ndarray): The intersection of prediction and
            ground truth histogram on all classes.
        total_area_union (ndarray): The union of prediction and ground truth
            histogram on all classes.
        total_area_pred_label (ndarray): The prediction histogram on all
            classes.
        total_area_label (ndarray): The ground truth histogram on all classes.
        metrics (list[str] | str): Metrics to be evaluated, 'mIoU' and 'mDice'.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
     Returns:
        float: Overall accuracy on all images.
        ndarray: Per category accuracy, shape (num_classes, ).
        ndarray: Per category evaluation metrics, shape (num_classes, ).
    """
    if isinstance(metrics, str):
        metrics = [metrics]
    allowed_metrics = ['mIoU', 'mDice', 'mFscore']
    if not set(metrics).issubset(set(allowed_metrics)):
        raise KeyError('metrics {} is not supported'.format(metrics))

    all_acc = total_area_intersect.sum() / total_area_label.sum()
    ret_metrics = OrderedDict({'aAcc': all_acc})
    for metric in metrics:
        if metric == 'mIoU':
            iou = total_area_intersect / total_area_union
            acc = total_area_intersect / total_area_label
            ret_metrics['IoU'] = iou
            ret_metrics['Acc'] = acc
        elif metric == 'mDice':
            print(f'\033[31\nper_img_mdice:{per_img_mdice}\033[39m')
            if per_img_mdice is None:
                dice = 2 * total_area_intersect / (
                    total_area_pred_label + total_area_label)
            else:
                dice = per_img_mdice

            acc = total_area_intersect / total_area_label
            ret_metrics['Dice'] = dice
            ret_metrics['Acc'] = acc
        elif metric == 'mFscore':
            precision = total_area_intersect / total_area_pred_label
            recall = total_area_intersect / total_area_label
            f_value = torch.tensor(
                [f_score(x[0], x[1], beta) for x in zip(precision, recall)])
            ret_metrics['Fscore'] = f_value
            ret_metrics['Precision'] = precision
            ret_metrics['Recall'] = recall

    ret_metrics = {
        metric: value.numpy()
        for metric, value in ret_metrics.items()
    }
    if nan_to_num is not None:
        ret_metrics = OrderedDict({
            metric: np.nan_to_num(metric_value, nan=nan_to_num)
            for metric, metric_value in ret_metrics.items()
        })
    return ret_metrics

EOF

In [ ]:
%%bash
cat << EOF > /content/mmsegmentation/mmseg/core/evaluation/metrics.py
# Copyright (c) OpenMMLab. All rights reserved.
from collections import OrderedDict

import mmcv
import numpy as np
import torch


def f_score(precision, recall, beta=1):
    """calculate the f-score value.

    Args:
        precision (float | torch.Tensor): The precision value.
        recall (float | torch.Tensor): The recall value.
        beta (int): Determines the weight of recall in the combined score.
            Default: False.

    Returns:
        [torch.tensor]: The f-score value.
    """
    score = (1 + beta**2) * (precision * recall) / (
        (beta**2 * precision) + recall)
    return score


def intersect_and_union(pred_label,
                        label,
                        num_classes,
                        ignore_index,
                        label_map=dict(),
                        reduce_zero_label=False):
    """Calculate intersection and Union.

    Args:
        pred_label (ndarray | str): Prediction segmentation map
            or predict result filename.
        label (ndarray | str): Ground truth segmentation map
            or label filename.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        label_map (dict): Mapping old labels to new labels. The parameter will
            work only when label is str. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. The parameter will
            work only when label is str. Default: False.

     Returns:
         torch.Tensor: The intersection of prediction and ground truth
            histogram on all classes.
         torch.Tensor: The union of prediction and ground truth histogram on
            all classes.
         torch.Tensor: The prediction histogram on all classes.
         torch.Tensor: The ground truth histogram on all classes.
    """

    if isinstance(pred_label, str):
        pred_label = torch.from_numpy(np.load(pred_label))
    else:
        pred_label = torch.from_numpy((pred_label))

    if isinstance(label, str):
        label = torch.from_numpy(
            mmcv.imread(label, flag='unchanged', backend='pillow'))
    else:
        label = torch.from_numpy(label)

    if label_map is not None:
        for old_id, new_id in label_map.items():
            label[label == old_id] = new_id
    if reduce_zero_label:
        label[label == 0] = 255
        label = label - 1
        label[label == 254] = 255

    mask = (label != ignore_index)
    pred_label = pred_label[mask]
    label = label[mask]

    intersect = pred_label[pred_label == label]
    area_intersect = torch.histc(
        intersect.float(), bins=(num_classes), min=0, max=num_classes - 1)
    area_pred_label = torch.histc(
        pred_label.float(), bins=(num_classes), min=0, max=num_classes - 1)
    area_label = torch.histc(
        label.float(), bins=(num_classes), min=0, max=num_classes - 1)
    area_union = area_pred_label + area_label - area_intersect
    return area_intersect, area_union, area_pred_label, area_label


def total_intersect_and_union(results,
                              gt_seg_maps,
                              num_classes,
                              ignore_index,
                              label_map=dict(),
                              reduce_zero_label=False,
                              ):
    """Calculate Total Intersection and Union.

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str] | Iterables): list of ground
            truth segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.

     Returns:
         ndarray: The intersection of prediction and ground truth histogram
             on all classes.
         ndarray: The union of prediction and ground truth histogram on all
             classes.
         ndarray: The prediction histogram on all classes.
         ndarray: The ground truth histogram on all classes.
    """

    print('excuting <total_intersect_and_union>')
    total_area_intersect = torch.zeros((num_classes, ), dtype=torch.float64)
    total_area_union = torch.zeros((num_classes, ), dtype=torch.float64)
    total_area_pred_label = torch.zeros((num_classes, ), dtype=torch.float64)
    total_area_label = torch.zeros((num_classes, ), dtype=torch.float64)
    per_img_mdice = torch.zeros((num_classes, ), dtype=torch.float64)

    cnt = 0
    for result, gt_seg_map in zip(results, gt_seg_maps):
        area_intersect, area_union, area_pred_label, area_label = \
            intersect_and_union(
                result, gt_seg_map, num_classes, ignore_index,
                label_map, reduce_zero_label)
        total_area_intersect += area_intersect
        total_area_union += area_union
        total_area_pred_label += area_pred_label
        total_area_label += area_label

        dice_coefficient = 2 * area_intersect / (area_pred_label + area_label)
        per_img_mdice += dice_coefficient
        cnt += 1

    per_img_mdice /= cnt
    return total_area_intersect, total_area_union, total_area_pred_label, \
        total_area_label, per_img_mdice


def mean_iou(results,
             gt_seg_maps,
             num_classes,
             ignore_index,
             nan_to_num=None,
             label_map=dict(),
             reduce_zero_label=False):
    """Calculate Mean Intersection and Union (mIoU)

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str]): list of ground truth
            segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.

     Returns:
        dict[str, float | ndarray]:
            <aAcc> float: Overall accuracy on all images.
            <Acc> ndarray: Per category accuracy, shape (num_classes, ).
            <IoU> ndarray: Per category IoU, shape (num_classes, ).
    """
    iou_result = eval_metrics(
        results=results,
        gt_seg_maps=gt_seg_maps,
        num_classes=num_classes,
        ignore_index=ignore_index,
        metrics=['mIoU'],
        nan_to_num=nan_to_num,
        label_map=label_map,
        reduce_zero_label=reduce_zero_label)
    return iou_result


def mean_dice(results,
              gt_seg_maps,
              num_classes,
              ignore_index,
              nan_to_num=None,
              label_map=dict(),
              reduce_zero_label=False):
    """Calculate Mean Dice (mDice)

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str]): list of ground truth
            segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.

     Returns:
        dict[str, float | ndarray]: Default metrics.
            <aAcc> float: Overall accuracy on all images.
            <Acc> ndarray: Per category accuracy, shape (num_classes, ).
            <Dice> ndarray: Per category dice, shape (num_classes, ).
    """

    dice_result = eval_metrics(
        results=results,
        gt_seg_maps=gt_seg_maps,
        num_classes=num_classes,
        ignore_index=ignore_index,
        metrics=['mDice'],
        nan_to_num=nan_to_num,
        label_map=label_map,
        reduce_zero_label=reduce_zero_label)
    return dice_result


def mean_fscore(results,
                gt_seg_maps,
                num_classes,
                ignore_index,
                nan_to_num=None,
                label_map=dict(),
                reduce_zero_label=False,
                beta=1):
    """Calculate Mean F-Score (mFscore)

    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str]): list of ground truth
            segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.
        beta (int): Determines the weight of recall in the combined score.
            Default: False.


     Returns:
        dict[str, float | ndarray]: Default metrics.
            <aAcc> float: Overall accuracy on all images.
            <Fscore> ndarray: Per category recall, shape (num_classes, ).
            <Precision> ndarray: Per category precision, shape (num_classes, ).
            <Recall> ndarray: Per category f-score, shape (num_classes, ).
    """
    fscore_result = eval_metrics(
        results=results,
        gt_seg_maps=gt_seg_maps,
        num_classes=num_classes,
        ignore_index=ignore_index,
        metrics=['mFscore'],
        nan_to_num=nan_to_num,
        label_map=label_map,
        reduce_zero_label=reduce_zero_label,
        beta=beta)
    return fscore_result


def eval_metrics(results,
                 gt_seg_maps,
                 num_classes,
                 ignore_index,
                 metrics=['mIoU'],
                 nan_to_num=None,
                 label_map=dict(),
                 reduce_zero_label=False,
                 beta=1):
    """Calculate evaluation metrics
    Args:
        results (list[ndarray] | list[str]): List of prediction segmentation
            maps or list of prediction result filenames.
        gt_seg_maps (list[ndarray] | list[str] | Iterables): list of ground
            truth segmentation maps or list of label filenames.
        num_classes (int): Number of categories.
        ignore_index (int): Index that will be ignored in evaluation.
        metrics (list[str] | str): Metrics to be evaluated, 'mIoU' and 'mDice'.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
        label_map (dict): Mapping old labels to new labels. Default: dict().
        reduce_zero_label (bool): Whether ignore zero label. Default: False.
     Returns:
        float: Overall accuracy on all images.
        ndarray: Per category accuracy, shape (num_classes, ).
        ndarray: Per category evaluation metrics, shape (num_classes, ).
    """

    total_area_intersect, total_area_union, total_area_pred_label, \
        total_area_label, per_img_mdice = total_intersect_and_union(
            results, gt_seg_maps, num_classes, ignore_index, label_map,
            reduce_zero_label)
    ret_metrics = total_area_to_metrics(total_area_intersect, total_area_union,
                                        total_area_pred_label,
                                        total_area_label,
                                        per_img_mdice,
                                        metrics, nan_to_num,
                                        beta)

    return ret_metrics


def pre_eval_to_metrics(pre_eval_results,
                        metrics=['mIoU'],
                        nan_to_num=None,
                        beta=1):
    """Convert pre-eval results to metrics.

    Args:
        pre_eval_results (list[tuple[torch.Tensor]]): per image eval results
            for computing evaluation metric
        metrics (list[str] | str): Metrics to be evaluated, 'mIoU' and 'mDice'.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
     Returns:
        float: Overall accuracy on all images.
        ndarray: Per category accuracy, shape (num_classes, ).
        ndarray: Per category evaluation metrics, shape (num_classes, ).
    """

    # convert list of tuples to tuple of lists, e.g.
    # [(A_1, B_1, C_1, D_1), ...,  (A_n, B_n, C_n, D_n)] to
    # ([A_1, ..., A_n], ..., [D_1, ..., D_n])
    print('\nexcuting <pre_eval_to_metrics>')
    pre_eval_results = tuple(zip(*pre_eval_results))
    assert len(pre_eval_results) == 4

    total_area_intersect = sum(pre_eval_results[0])
    total_area_union = sum(pre_eval_results[1])
    total_area_pred_label = sum(pre_eval_results[2])
    total_area_label = sum(pre_eval_results[3])
    per_img_mdice = None  # sum(pre_eval_results[4])

    ret_metrics = total_area_to_metrics(total_area_intersect, total_area_union,
                                        total_area_pred_label,
                                        total_area_label,
                                        per_img_mdice,
                                        metrics, nan_to_num,
                                        beta)

    return ret_metrics


def total_area_to_metrics(total_area_intersect,
                          total_area_union,
                          total_area_pred_label,
                          total_area_label,
                          per_img_mdice,
                          metrics=['mIoU'],
                          nan_to_num=None,
                          beta=1):
    """Calculate evaluation metrics
    Args:
        total_area_intersect (ndarray): The intersection of prediction and
            ground truth histogram on all classes.
        total_area_union (ndarray): The union of prediction and ground truth
            histogram on all classes.
        total_area_pred_label (ndarray): The prediction histogram on all
            classes.
        total_area_label (ndarray): The ground truth histogram on all classes.
        metrics (list[str] | str): Metrics to be evaluated, 'mIoU' and 'mDice'.
        nan_to_num (int, optional): If specified, NaN values will be replaced
            by the numbers defined by the user. Default: None.
     Returns:
        float: Overall accuracy on all images.
        ndarray: Per category accuracy, shape (num_classes, ).
        ndarray: Per category evaluation metrics, shape (num_classes, ).
    """
    if isinstance(metrics, str):
        metrics = [metrics]
    allowed_metrics = ['mIoU', 'mDice', 'mFscore']
    if not set(metrics).issubset(set(allowed_metrics)):
        raise KeyError('metrics {} is not supported'.format(metrics))

    all_acc = total_area_intersect.sum() / total_area_label.sum()
    ret_metrics = OrderedDict({'aAcc': all_acc})
    for metric in metrics:
        if metric == 'mIoU':
            iou = total_area_intersect / total_area_union
            acc = total_area_intersect / total_area_label
            ret_metrics['IoU'] = iou
            ret_metrics['Acc'] = acc
        elif metric == 'mDice':
            print(f'\033[31\nper_img_mdice:{per_img_mdice}\033[39m')
            if per_img_mdice is None:
                dice = 2 * total_area_intersect / (
                    total_area_pred_label + total_area_label)
            else:
                dice = per_img_mdice

            acc = total_area_intersect / total_area_label
            ret_metrics['Dice'] = dice
            ret_metrics['Acc'] = acc
        elif metric == 'mFscore':
            precision = total_area_intersect / total_area_pred_label
            recall = total_area_intersect / total_area_label
            f_value = torch.tensor(
                [f_score(x[0], x[1], beta) for x in zip(precision, recall)])
            ret_metrics['Fscore'] = f_value
            ret_metrics['Precision'] = precision
            ret_metrics['Recall'] = recall

    ret_metrics = {
        metric: value.numpy()
        for metric, value in ret_metrics.items()
    }
    if nan_to_num is not None:
        ret_metrics = OrderedDict({
            metric: np.nan_to_num(metric_value, nan=nan_to_num)
            for metric, metric_value in ret_metrics.items()
        })
    return ret_metrics

EOF

## Prepare configs

In [ ]:
%%bash

cat << EOF > /content/mmsegmentation/configs/deeplabv3plus_r50-d8_512x512_hubmap.py

#-------------------------------------------------------------------------
# model settings
norm_cfg = dict(type='GN', num_groups=32, requires_grad=True)
model = dict(
    type='EncoderDecoder',
    backbone=dict(
        type='MixVisionTransformer',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segformer/mit_b3_20220624-13b1141c.pth'
            ),
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 4, 18, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1),
    decode_head=dict(
        type='SegformerHead',
        in_channels=[64, 128, 320, 512],
        in_index=[0, 1, 2, 3],
        channels=256,
        dropout_ratio=0.1,
        num_classes=2,
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=[
            dict(type='CrossEntropyLoss', loss_name='loss_ce', loss_weight=1),
            dict(type='LovaszLoss', loss_name='loss_lovasz', per_image=True, loss_weight=3),
        ]
        ),
    train_cfg=dict(),
    test_cfg=dict(mode='whole'))



#-------------------------------------------------------------------------
# dataset settings

dataset_type = 'HubmapDataset'
data_root = '/content'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
crop_size = (1024, 1024)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='OrgansDataAug'),
    dict(type='Resize', img_scale=[(480,480),(1600, 1600)], keep_ratio=False, ratio_range=None),
    dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.5), #cat_max_ratio=0.75
    dict(type='RandomFlip', direction='horizontal', prob=0),
    #dict(type='RandomRotate', degree=180, prob=0.7),
    dict(type='SaveOverlay', save_root_dir='/content', save_num=500, no_overlay=True),
    dict(type='Convert2Class1'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024), #画像のリサイズ(モデルのinputサイズではない)
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=True,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


data = dict(
    samples_per_gpu=1, #batch
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir=[
            'hubmap_multi_2000x2000/images',
            'hubmap_multi_2000x2000/images_stained_with_hubmap1',
            'hubmap_multi_2000x2000/images_stained_with_hubmap2',
            'hubmap_multi_2000x2000/images_stained_with_sample1',
            'hubmap_multi_2000x2000/images_stained_with_test',
            'external_spleen_v2/image',
            'external_lung_v1/image'
        ],
        ann_dir=[
            'hubmap_multi_2000x2000/lung_refined_masks',
            'hubmap_multi_2000x2000/lung_refined_masks',
            'hubmap_multi_2000x2000/lung_refined_masks',
            'hubmap_multi_2000x2000/lung_refined_masks',
            'hubmap_multi_2000x2000/lung_refined_masks',
            'external_spleen_v2/mask',
            'external_lung_v1/mask_0.3'
        ],
        img_suffix='.png',
        seg_map_suffix='.png',
        split= [
            '/content/hubmap_multi_2000x2000/ImageSets/Segmentation/trainval.txt',
            '/content/hubmap_multi_2000x2000/ImageSets/Segmentation/trainval.txt',
            '/content/hubmap_multi_2000x2000/ImageSets/Segmentation/trainval.txt',
            '/content/hubmap_multi_2000x2000/ImageSets/Segmentation/trainval.txt',
            '/content/hubmap_multi_2000x2000/ImageSets/Segmentation/trainval.txt',
            '/content/external_spleen_v2/trainval.txt',
            '/content/external_lung_v1/external_lung.txt'
        ],
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='hubmap_multi_2000x2000/images_stained_with_hubmap1',
        ann_dir='hubmap_multi_2000x2000/lung_refined_masks_2class',
        img_suffix='.png',
        seg_map_suffix='.png',
        split='/content/hubmap_multi_2000x2000/ImageSets/Segmentation/val_fold0.txt',
        pipeline=test_pipeline),
    test=dict()
    )

#-------------------------------------------------------------------------
# '../_base_/schedules/schedule_20k.py'

# optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0005)
optimizer_config = dict()
# lr_config = dict(warmup='linear', warmup_iters=500, by_epoch=False, 
#                  policy='poly', power=0.9, min_lr=0.001)

optimizer = dict(
    type='AdamW',
    lr=0.00006,#0.00006
    betas=(0.9, 0.999),
    weight_decay=0.01,
    paramwise_cfg=dict(
        custom_keys={
            'pos_block': dict(decay_mult=0.),
            'norm': dict(decay_mult=0.),
            'head': dict(lr_mult=10.)
        }))

lr_config = dict(
    policy='poly',
    warmup='linear',
    warmup_iters=1500,
    warmup_ratio=1e-6,
    power=1.0,
    min_lr=0,
    by_epoch=False)


# runtime settings
runner = dict(type='IterBasedRunner', max_iters=45000)
checkpoint_config = dict(by_epoch=False, interval=45000) #max_keep_ckpts
evaluation = dict(interval=3000, metric=['mDice', 'mIoU'], pre_eval=False) #save_best='mIoU'

#-------------------------------------------------------------------------
# _base_/default_runtime.py

log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook', by_epoch=False),

        dict(type='MMSegWandbHook',
                init_kwargs=dict(project="HubMap-960-refined-val",
                                 name=f'mit-b3_1024',
                                 config={'config':'deeplabv3plus_r50-d8_512x512_hubmap.py',
                                         'comment':'No comment',
                                         'dataset_type': dataset_type,
                                         'model': model,
                                         'crop_size': crop_size,
                                         'train_pipeline': train_pipeline,
                                         'test_pipeline': test_pipeline,
                                         'optimizer':optimizer,
                                         'lr_config':lr_config,
                                         'runner': runner,
                                         'checkpoint_config': checkpoint_config,
                                         'evaluation':evaluation,
                                         'data': data,
                                         },
                                 #group='',
                                 entity=None),
                interval=3000,
                log_checkpoint=True,
                log_checkpoint_metadata=True,
                num_eval_images=100)
    ]
    )

dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
cudnn_benchmark = True



#-------------------------------------------------------------------------
work_dir='/content/hubmap_training'

from mmseg.apis import set_random_seed
set_random_seed(0, deterministic=False)

# 必須
seed = 0
gpu_ids = range(1)
device='cuda'

EOF

## Train

In [ ]:
%cd /content/mmsegmentation

config='configs/deeplabv3plus_r50-d8_512x512_hubmap.py'

!rm -r /content/overlay
!python tools/train.py {config}

ストリーミング出力は最後の 5000 行に切り捨てられました。
2022-09-19 01:08:23,671 - mmseg - INFO - Loaded 351 images
[OrgansDataAug] organ:kidney transforms:[Crop(always_apply=False, p=0.7, x_min=100, y_min=100, x_max=1900, y_max=1900), Flip(always_apply=False, p=1), RandomRotate90(always_apply=False, p=1), HueSaturationValue(always_apply=False, p=1, hue_shift_limit=(-20, 20), sat_shift_limit=(-20, 20), val_shift_limit=(-20, 20)), RandomGamma(always_apply=False, p=1, gamma_limit=(50.0, 300.0), eps=None), RandomBrightness(always_apply=False, p=1, limit=(-0.2, 0.1))]
[OrgansDataAug] organ:largeintestine transforms:[Crop(always_apply=False, p=0.7, x_min=100, y_min=100, x_max=1900, y_max=1900), Flip(always_apply=False, p=1), RandomRotate90(always_apply=False, p=1), HueSaturationValue(always_apply=False, p=1, hue_shift_limit=(-20, 20), sat_shift_limit=(-20, 20), val_shift_limit=(-20, 20)), RandomGamma(always_apply=False, p=1, gamma_limit=(50.0, 300.0), eps=None), RandomBrightness(always_apply=False, p=1, limit=(-0.2